---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [7]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [8]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [29]:
def degree_distribution(G):
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes( G)) for i in degree_values]
    return histogram

In [30]:
degree_distribution(P1_Graphs[2])

[0.0026666666666666666, 0.124, 0.756, 0.10533333333333333, 0.012]

In [31]:
nx.average_clustering(P1_Graphs[1])

0.5642419635919628

In [32]:
nx.average_shortest_path_length(P1_Graphs[1])

5.089871871871872

In [33]:
for G in P1_Graphs:
    print(nx.average_clustering(G), nx.average_shortest_path_length(G), len(degree_distribution(G)))


0.03167539146454044 4.099161161161161 29
0.5642419635919628 5.089871871871872 7
0.4018222222222227 9.378702269692925 5
0.03780379975223251 3.1048046283934134 37
0.0033037037037037037 5.0785509568313305 9


In [34]:
def graph_identification():
    methods = []
    for G in P1_Graphs:
        clustering = nx.average_clustering(G)
        shortest_path = nx.average_shortest_path_length(G)
        degree_hist = degree_distribution(G)
        if len(degree_hist) > 10:
            methods.append('PA')
        elif clustering < 0.1:
            methods.append('SW_H')
        else: 
            methods.append('SW_L')
    return methods
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [35]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [37]:
def salary_predictions():
    
    df = pd.DataFrame(index=G.nodes())
    df['departement'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['management'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['cc'] = pd.Series(nx.closeness_centrality(G))
    df['bc'] = pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints=False))
    df['pr'] = pd.Series(nx.pagerank(G, alpha=0.8))
    df['hub'] = pd.Series(nx.hits(G)[0])
    # df['autho'] = pd.Series(nx.hits(G)[1])
    # df.head()
    
    y = df['management']
    y_train = y[y.notnull()]

    X = df.drop(['management'], axis=1)
    X_train = X[df.management.notnull()]
    X_test = X[df.management.isnull()]

    from sklearn.ensemble import RandomForestClassifier 
    from sklearn.model_selection import cross_val_score

#     for depth in range(3,10):
#         rfc = RandomForestClassifier(n_estimators=100, max_depth=depth)
#         print(np.mean(cross_val_score(rfc, X_train, y_train, cv=5, scoring='roc_auc')))


    rfc = RandomForestClassifier(n_estimators=100, max_depth=4)
    rfc.fit(X_train, y_train)

    prob = rfc.predict_proba(X_test)
    ans = pd.Series([x[1] for x in prob], index = X_test.index)
    return ans 
salary_predictions()

1       0.134356
2       0.908748
5       0.999343
8       0.156741
14      0.119261
18      0.130365
27      0.142867
30      0.426113
31      0.361589
34      0.073848
37      0.075934
40      0.140411
45      0.154496
54      0.399771
55      0.391505
60      0.242145
62      0.997787
65      0.904923
77      0.129739
79      0.162716
97      0.024780
101     0.027988
103     0.345923
108     0.154885
113     0.185375
122     0.021616
141     0.355324
142     0.996735
144     0.051962
145     0.429928
          ...   
913     0.018684
914     0.023033
915     0.012631
918     0.091981
923     0.037967
926     0.044522
931     0.017964
934     0.012660
939     0.010816
944     0.014851
945     0.037967
947     0.084276
950     0.183413
951     0.017659
953     0.014335
959     0.010441
962     0.009587
963     0.357492
968     0.050679
969     0.052893
974     0.043168
984     0.013380
987     0.048103
989     0.049271
991     0.079471
992     0.009818
994     0.014851
996     0.0140

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [38]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [39]:
def new_connections_predictions():
    
    df = future_connections
    df['pa'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    df['cn'] = df.index.map(lambda person: len(list(nx.common_neighbors(G, person[0], person[1]))))
    df['jc'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['rai'] = [i[2] for i in nx.resource_allocation_index(G, df.index)]
    df['aai'] = [i[2] for i in nx.adamic_adar_index(G, df.index)]
    
    y = df['Future Connection']
    y_train = y[y.notnull()]

    X = df.drop(['Future Connection'], axis=1)
    X_train = X[y.notnull()]
    X_test = X[y.isnull()]
    
    from sklearn.ensemble import RandomForestClassifier 
    from sklearn.model_selection import cross_val_score

#     for depth in range(3,10):
#         rfc = RandomForestClassifier(n_estimators=10, max_depth=depth)
#         print(np.mean(cross_val_score(rfc, X_train, y_train, cv=3, scoring='roc_auc')))
    
    rfc = RandomForestClassifier(n_estimators=10, max_depth=5)
    rfc.fit(X_train, y_train)

    prob = rfc.predict_proba(X_test)
    ans = pd.Series([x[1] for x in prob], index = X_test.index)

    return ans 
new_connections_predictions()

(107, 348)    0.034757
(542, 751)    0.013162
(20, 426)     0.524483
(50, 989)     0.013162
(942, 986)    0.013162
(324, 857)    0.013162
(13, 710)     0.184086
(19, 271)     0.143400
(319, 878)    0.013162
(659, 707)    0.013162
(49, 843)     0.013162
(208, 893)    0.013162
(377, 469)    0.013162
(405, 999)    0.020198
(129, 740)    0.013162
(292, 618)    0.074169
(239, 689)    0.013162
(359, 373)    0.013162
(53, 523)     0.033065
(276, 984)    0.013162
(202, 997)    0.013162
(604, 619)    0.078622
(270, 911)    0.013162
(261, 481)    0.060547
(200, 450)    0.915723
(213, 634)    0.013162
(644, 735)    0.083134
(346, 553)    0.013162
(521, 738)    0.013162
(422, 953)    0.020198
                ...   
(672, 848)    0.013162
(28, 127)     0.977153
(202, 661)    0.013162
(54, 195)     0.996495
(295, 864)    0.013162
(814, 936)    0.013162
(839, 874)    0.013162
(139, 843)    0.013162
(461, 544)    0.013162
(68, 487)     0.013162
(622, 932)    0.013162
(504, 936)    0.016786
(479, 528) 